### Here we make final preparation before simulation of epidemics in the city

In [67]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from shapely.geometry import Point
from tqdm import tqdm
from geopy.distance import great_circle
import time as time
import random
from scipy.stats import binom

In [68]:
city_name = 'samara'

input_path = '../data/' + city_name + '/people_' + city_name + '_2023_splitted_workplaces.txt'

people = pd.read_csv(input_path, sep='\t', index_col=0)

def move_school_id_to_work_id(row):
    value = 0
    if row['school_id'] != -1:
        value = row['school_id']
    else:
        value = row['work_id']
    return value


people['work_id'] = people.apply(move_school_id_to_work_id, axis  = 1)
people_processed = people.drop(columns=['Unnamed: 0', 'race', 'relate'])
people_processed.head()

,sp_id,sp_hh_id,age,sex,school_id,work_id
0,704990,1,30,F,-1,1
2,680160,2,26,F,-1,1
3,974991,2,60,F,-1,1
5,231508,2,35,M,-1,1
6,170205,3,28,M,-1,1


In [69]:
def change_work_to_X(row):
    value = 0
    if (row['work_id'] == -1):
        value = 'X'
    else:
        value = row['work_id']
    return value

def change_school_to_X(row):
    value = 0
    if (row['school_id'] == -1):
        value = 'X'
    else:
        value = row['school_id']
    return value
    
    
people_processed['work_id'] = people_processed.apply(change_work_to_X, axis  = 1)
people_processed['school_id'] = people_processed.apply(change_school_to_X, axis  = 1)
people_processed.sort_index(inplace=True)

# len(people_processed[people_processed['work_id'] == 'X'])

##### And also we need to create schools.json file for model with school number as key and values are sp_id's

In [70]:
school_dict = dict()


for idx in tqdm(range(len(people_processed))):
    row = people_processed.iloc[idx]
    school_id = str(row['school_id'])
    if (school_id != 'X' and school_id != -1):
        if (school_id in school_dict):
            school_dict[school_id].append(int(idx))
        else:
            school_dict[school_id] = [int(idx)]

# del school_dict['X']

100%|██████████| 1158519/1158519 [00:53<00:00, 21631.92it/s]


In [71]:
people_processed

,sp_id,sp_hh_id,age,sex,school_id,work_id
0,704990,1,30,F,X,1
1,1,2,0,M,X,X
2,680160,2,26,F,X,1
3,974991,2,60,F,X,1
4,1041462,2,67,F,X,X
...,...,...,...,...,...,...
1158514,994470,452020,61,F,X,X
1158515,731017,452020,32,F,X,74185
1158516,721629,452020,31,F,X,74185
1158517,396712,452020,54,M,X,74185


In [73]:
import json

with open('../data/' + city_name + '/schools_' + city_name + '_final.json', 'w') as fp:
    json.dump(school_dict, fp)
    
people_processed = people_processed.drop(columns=['school_id'])
people_processed.head(15)
people_processed.to_csv('../data/' + city_name + '/people_' + city_name + '_2023_final.txt', sep='\t')


KeyError: "['school_id'] not found in axis"